In [1]:
import pandas as pd

df = pd.read_csv('../../../python-machine-learning-book-3rd-edition-master/ch08/movie_data.csv', encoding='utf-8')
df.shape

(50000, 2)

## Bag-of-Words

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two'
])
bag = count.fit_transform(docs)

print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [3]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [4]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [5]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [6]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [7]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [8]:
df['review'] = df['review'].apply(preprocessor)

In [9]:
def tokenizer(text):
    return text.split()

tokenizer('return like running and thus they run')

['return', 'like', 'running', 'and', 'thus', 'they', 'run']

In [10]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('return like running and thus they run')

['return', 'like', 'run', 'and', 'thu', 'they', 'run']

In [12]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sekine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner like running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [15]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    strip_accents=None,
    lowercase=False,
    preprocessor=None
)
param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 10.0, 100.0]
    }, {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'vect__use_idf':[False],
        'vect__norm':[None],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 10.0, 100.0]
    }
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(random_state=0, solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(
    lr_tfidf, param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/sekine/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 37.9min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

In [17]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_: .3f}')

clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test, y_test): .3f}')

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7fba401e6820>}
CV Accuracy:  0.897
Test Accuracy:  0.899


## Out-of-core learning

In [18]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [19]:
next(stream_docs(path='../../../python-machine-learning-book-3rd-edition-master/ch08/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [20]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [21]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(
    decode_error='ignore',
    n_features=2 ** 21,
    preprocessor=None,
    tokenizer=tokenizer
)
clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path='../../../python-machine-learning-book-3rd-edition-master/ch08/movie_data.csv')

In [22]:
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)

In [23]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test): .3f}')

Accuracy:  0.868


## Topic model

In [24]:
df.shape

(50000, 2)

In [26]:
count = CountVectorizer(
    stop_words='english',
    max_df=.1,
    max_features=5000
)
X = count.fit_transform(df['review'].values)

In [27]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
    n_components=10,
    random_state=114,
    learning_method='batch'
)
X_topics = lda.fit_transform(X)

In [28]:
lda.components_.shape

(10, 5000)

In [29]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {topic_idx + 1}')
    print(' '.join([feature_names[i] for i in topic.argsort() [:- n_top_words - 1: - 1]]))

Topic 1
original animation disney king japanese
Topic 2
family war father woman mother
Topic 3
comedy play music musical plays
Topic 4
guy action script stupid worst
Topic 5
human cinema art set style
Topic 6
horror killer gore blood house
Topic 7
school kids town high black
Topic 8
role performance performances actor wonderful
Topic 9
worst minutes awful terrible stupid
Topic 10
series tv book episode dvd


In [30]:
topic1 = X_topics[:, 0].argsort()[::-1]

for iter_idx, movie_idx in enumerate(topic1[:3]):
    print('\nTopic1 movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


Topic1 movie #1:
oz is the greatest show ever mad full stop oz is the greatest show ever mad full stop oz is the greatest show ever mad full stop oz is the greatest show ever mad full stop oz is the greatest show ever mad full stop oz is the greatest show ever mad full stop oz is the greatest show ever mad full stop ...

Topic1 movie #2:
a respectable royal rumble event1 edge vs shawn michaels7 5 10 a very strong opener edge s heel performance was sublime as it was during the rumble event overall id say edges night 2 undertaker vs heidenreich casket match7 10 a lot of people hated this rivalry though i liked it i thought heidenreich ...

Topic1 movie #3:
fire and ice is an animated film set in a fantasy world the film is about a village that is destroyed by a giant glacier which is the home of the evil ice lord named nekron the only survivor of the village is a young man named larn who sets out to avenge those who were killed by the glacier the ice  ...
